In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
using_collab = True

if using_collab :
  from google.colab import drive
  drive.mount('/content/drive')

file_path = "/content/drive/MyDrive" #file_path를 설정해주세요.

Mounted at /content/drive


In [4]:
df = pd.read_csv(file_path + "/KISTI/rawdata/raw_dataset_kor.csv", encoding='cp949')

In [5]:
df = df.iloc[:,1:]

In [6]:
df = df.replace({np.nan: None})

In [7]:
df.head(3)

,APPLICATION_NUMBER,TITLE,DISC_TECH_SOLUTION,DISC_ADV_EFFECT,DISC_TECH_PROBLEM,ABSTRACT
0,1020187012229,에러들을 표현하기 위한 방법들 및 시스템들,[ NULL ],[ NULL ],[ NULL ],None
1,1020187012203,"스프링 유닛, 스프링 어큐뮬레이터, 및 액추에이터",[ NULL ],[ NULL ],[ NULL ],None
2,1020187012108,나노입자 및 이의 제조 방법,[ NULL ],[ NULL ],[ NULL ],None


In [8]:
df.iloc[0]["DISC_TECH_SOLUTION"]

'[ NULL ] '

### Abs. => Title

In [9]:
INST = "다음 이 발명에 대한 요약을 읽고, 이 발명의 제목을 예측하시오."
# INPUT = ABSTRACT
# OUTPUT = TITLE

In [10]:
df_tit = df.drop_duplicates(subset=['TITLE'])

In [11]:
len(df_tit)

43809

In [12]:
task1 = []
for i, row in df_tit.iterrows():
    if row['ABSTRACT'] is not None:
        tmp_dict = {}
        tmp_dict['instruction'] = INST
        tmp_dict['input'] = row['ABSTRACT']
        tmp_dict['output'] = f"이 발명의 제목은 {row['TITLE']} 입니다."
        task1.append(tmp_dict)

In [13]:
len(task1)

5808

### DISC_TECH_SOLUTION, DISC_ADV_EFFECT, DISC_TECH_PROBLEM => TITLE

In [14]:
df_tit = df_tit.replace({'[ NULL ] ': None})

In [15]:
task2 = []
INST = "다음 이 발명에 대한 설명을 읽고, 이 발명의 제목을 예측하시오."

for i, row in df_tit.iterrows():
    tmp_dict = {}
    INPUT = ""
    if row['DISC_TECH_PROBLEM'] is not None:
        INPUT = INPUT + "이 발명이 풀고자 하는 문제 : " + row['DISC_TECH_PROBLEM']
    if row['DISC_TECH_SOLUTION'] is not None:
        INPUT = INPUT + "이 발명의 원리 : " + row['DISC_TECH_SOLUTION']
    if row['DISC_ADV_EFFECT'] is not None:
        INPUT = INPUT + "이 발명의 효과 : " + row['DISC_ADV_EFFECT']
    OUTPUT = f"이 발명의 제목은 {row['TITLE']} 입니다."

    if INPUT != "":
        tmp_dict['instruction'] = INST
        tmp_dict['input'] = INPUT
        tmp_dict['output'] = OUTPUT
        task2.append(tmp_dict)

### DISC_TECH_SOLUTION, DISC_ADV_EFFECT, DISC_TECH_PROBLEM => TITLE, Abs

In [16]:
df_tit = df_tit[~df_tit['ABSTRACT'].isna()].reset_index(drop=True)

In [17]:
task3 = []
INST = "다음 이 발명에 대한 설명을 읽고, 이 발명의 제목을 예측하고 요약하시오."

for i, row in df_tit.iterrows():
    tmp_dict = {}
    INPUT = ""
    if row['DISC_TECH_PROBLEM'] is not None:
        INPUT = INPUT + "이 발명이 풀고자 하는 문제 : " + row['DISC_TECH_PROBLEM']
    if row['DISC_TECH_SOLUTION'] is not None:
        INPUT = INPUT + "이 발명의 원리 : " + row['DISC_TECH_SOLUTION']
    if row['DISC_ADV_EFFECT'] is not None:
        INPUT = INPUT + "이 발명의 효과 : " + row['DISC_ADV_EFFECT']

    OUTPUT = f'이 발명의 제목은 {row["TITLE"]} 이고, 위 발명은 다음과 같이 요약할 수 있습니다. {row["ABSTRACT"]}'

    if INPUT != "":
        tmp_dict['instruction'] = INST
        tmp_dict['input'] = INPUT
        tmp_dict['output'] = OUTPUT
        task3.append(tmp_dict)

In [18]:
task1_df = pd.DataFrame(task1)
task2_df = pd.DataFrame(task2)
task3_df = pd.DataFrame(task3)

In [19]:
task1_df['category']='task1'
task2_df['category']='task2'
task3_df['category']='task3'


task1_df['index']=task1_df.index
task2_df['index']=task2_df.index
task3_df['index']=task3_df.index



In [20]:
appended_table = pd.concat([task1_df, task2_df,task3_df], ignore_index=True)
appended_table['data'] = "kpa"

In [21]:
appended_table.head()

,instruction,input,output,category,index,data
0,"다음 이 발명에 대한 요약을 읽고, 이 발명의 제목을 예측하시오.","결석 제거에 자기 물질의 용도가 개시되며 , 상기 자기 물질은 나노입자 코어를 구성하며 , 표면 개질제 모노머가 개시제 및/또는 가교제를 통해 상기 나노입자 코어에 연결되어 있고 , 이로써 나노입자 쉘을 형성한다 . 만들어진 상기 나노입자는 요관 내부에서 결석을 코팅할 수 있고 외부 자기장의 작용 하에서 몸 내부에 잔존하는 작은 결석을 비침습적으로 및 신속히 제거할 수 있고 , 결석이 움직일 때 요관벽에 손상을 주지 않으며 , 동시에 위치시키기가 ( place ) 용이하며 결석 이동 ( calculus displacement ) 를 야기하지 않는다.",이 발명의 제목은 결석 제거에서 자성재료의 용도 입니다.,task1,0,kpa
1,"다음 이 발명에 대한 요약을 읽고, 이 발명의 제목을 예측하시오.","점착제 부착 편광판 ( 1 ) 은 , 요오드를 함유하는 폴리비닐알코올계 필름으로 이루어진 편광자 ( 11 ) 와 , 편광자의 제1 주면에 접합된 투명 보호 필름 ( 15 ) 과 , 편광자의 제2 주면에 접하여 설치된 전방면측 점착 시트 ( 20 ) 를 구비한다 . 전방면측 점착 시트는 , 적어도 2층의 점착제층이 적층된 적층 점착 시트이며 , 편광자에 접하여 배치된 제1 점착제층 ( 21 ) 과 , 편광자로부터 이격되어 배치된 제2 점착제층 ( 22 ) 을 구비한다 . 전방면측 점착 시트의 80℃에 있어서의 저장 탄성률 Ｇ ' Sub 80 /Sub 이 1×10 5 Pa 이하인 것이 바람직하고 , 제1 점착제층의 투습도는 150g/m 2 ㆍ24h 이하인 것이 바람직하다.",이 발명의 제목은 점착제 부착 편광판 및 화상 표시 장치 입니다.,task1,1,kpa
2,"다음 이 발명에 대한 요약을 읽고, 이 발명의 제목을 예측하시오.","본 발명은 열가소성 수지 조성물 , 이의 제조방법 및 이를 포함하는 성형품에 관한 것으로 , 보다 상세하게는 스티렌계 수지 , 실리콘-폴리카보네이트 , 초고분자량 실록산 화합물 및 폴리올레핀에 방향족 비닐 화합물 및 비닐시안 화합물이 그라프트된 그라프트 수지를 포함하는 열가소성 수지 조성물 , 이의 제조방법 및 이를 포함하는 성형품에 관한 것이다 . 본 발명에 따르면 , 시간경과나 외부 환경변화에 대해 수지의 충격강도 , 열변형 온도 등의 기계적 물성의 저하가 적고 , 가공성이 우수하면서 지속적으로 낮은 동마찰계수를 유지하여 마찰음이 적게 발생하는 열가소성 수지조성물을 제공하는 효과가 있다.","이 발명의 제목은 열가소성 수지 조성물, 이의 제조방법 및 이를 포함하는 성형품 입니다.",task1,2,kpa
3,"다음 이 발명에 대한 요약을 읽고, 이 발명의 제목을 예측하시오.","본 발명은 장방형의 판상형 전극으로 이루어진 양극 및 음극 사이에 분리막이 게재된 구조의 단위셀들이 적층되어 있는 전극조립체로서 , 전극조립체에서 단위셀들은 지면을 기준으로 수직으로 적층되어 있고 , 각각의 단위셀은 양극 및 음극을 포함하는 2k+1 ( 1≤k≤15 ) 개의 전극과 1개의 분리막으로 구성되어 있으며 , 상기 단위셀에서 양극과 음극은 교번 배열로 수직으로 적층되어 있고 , 분리막은 적층된 전극들 사이에 개재될 수 있도록 단위셀의 양측 단부들을 부분적으로 감싸면서 지그재그 형상으로 절곡되어 있으며 , 상기 단위셀의 외곽에 위치하는 일측 전극의 외측면에는 분리막이 위치하고 , 타측 전극의 외측면에는 분리막이 위치하지 않는 구조의 전극조립체에 대한 것이다.",이 발명의 제목은 절곡된 한 개의 분리막으로 이루어진 단위셀을 포함하고 있는 전극조립체 입니다.,task1,3,kpa
4,"다음 이 발명에 대한 요약을 읽고, 이 발명의 제목을 예측하시오.","본 발명은 탄소에 담지된 백금-전이 금속 합금 촉매의 제조방법에 관한 것으로서 , 더욱 상세하게는 탄소 담지 촉매에 탄소층 ( carbon layer ) 의 소재가 될 유기고분자를 코팅한 뒤 수소 결핍된 분위기에서 열처리하여 유기고분자를 탄소층으로 변환시켜 탄소층에 의해 열처리 과정에서 야기되는 촉매 입자의 크기 성장을 억제하도록 하였고 , 동시에 백금과 함께 담지된 전이 금속이 백금입자 속으로 확산되어 촉매의 표면에 백금 스킨층이 형성된 코어-쉘 구조의 촉매를 형성하도록 하였고 , 그리고 열처리 후에는 탄소층을 오존 처리를 통해 제거하여 촉매 표면이 전기화학적 반응을 할 수 있도록 하는 고활성 및 고내구성의 탄소 담지 백금 합금 촉매의 제조방법에 관한 것이다",이 발명의 제목은 탄소층 보호 코팅과 오존을 이용한 백금 합금 촉매의 제조방법 입니다.,task1,4,kpa


In [22]:
appended_table.to_csv(file_path+"/KISTI/dataset/kpa_task.csv")